In [ ]:
import sys
import pathlib
import numpy
import scipy.interpolate
import ipywidgets
import pandas
from h5py import File as h5open
from cycler import cycler
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import ticker
from matplotlib.legend_handler import HandlerTuple

In [ ]:
# find helpers and locate workdir
for parent in [pathlib.Path.cwd()] + list(pathlib.Path.cwd().parents):
    if parent.joinpath("modulus").is_dir():
        projdir = parent
        sys.path.insert(0, str(projdir.joinpath("modulus")))
        from helpers.utils import read_tensorboard_data  # pylint: disable=import-error
        break
else:
    raise FileNotFoundError("Couldn't find module `helpers`.")

# point workdir to the correct folder
workdir = projdir.joinpath("modulus", "cylinder-2d-re40")
petibmdir = projdir.joinpath("petibm", "cylinder-2d-re40")

In [ ]:
# unified figure style
pyplot.style.use(projdir.joinpath("resources", "figstyle"))

In [ ]:
def read_petibm_snapshot(workdir, time):
    """Read snapshots from PetIBM simulations.
    """
    assert float(int(time)) == float(time), f"Only supports integer time. Got {time}."

    # hard-coded simulation parameters; should match the config.yaml in the PetIBM case
    dt = 0.01
    step = int(time/dt+0.5)

    # get gridline and calculate coordinates
    with h5open(workdir.joinpath("output", "grid.h5"), "r") as h5file:
        coords = {
            r"$u$": numpy.meshgrid(h5file["u"]["x"][...], h5file["u"]["y"][...]),
            r"$v$": numpy.meshgrid(h5file["v"]["x"][...], h5file["v"]["y"][...]),
            r"$p$": numpy.meshgrid(h5file["p"]["x"][...], h5file["p"]["y"][...]),
            r"$\omega_z$": numpy.meshgrid(h5file["wz"]["x"][...], h5file["wz"]["y"][...]),
        }

    # get data
    with h5open(workdir.joinpath("output", f"{step:07d}.h5"), "r") as h5file:
        vals = {
            r"$u$": h5file["u"][...],
            r"$v$": h5file["v"][...],
            r"$p$": h5file["p"][...],
            r"$\omega_z$": h5file["wz"][...],
        }
    
    return coords, vals

In [ ]:
def read_pinn_snapshot(workdir, casename, time):
    """Read snapshots from PINN simulations.
    """
    with h5open(workdir.joinpath("outputs", casename).with_suffix(".h5"), "r") as h5file:
        coords = {
            r"$u$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$v$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$p$": (h5file["fields/x"][...], h5file["fields/y"][...]),
            r"$\omega_z$": (h5file["fields/x"][...], h5file["fields/y"][...]),
        }

        if not h5file.attrs["unsteady"]:
            time = "steady"

        vals = {
            r"$u$": h5file[f"fields/{time}/u"][...],
            r"$v$": h5file[f"fields/{time}/v"][...],
            r"$p$": h5file[f"fields/{time}/p"][...],
            r"$\omega_z$": h5file[f"fields/{time}/vorticity_z"][...],
        }
    
    return coords, vals

In [ ]:
def plot_single_snapshot(workdir, casename, time, petibmdir):
    """Plot a field of all cases at a single time.
    """

    # colored contour levels
    lvl1 = {
        r"$u$": numpy.linspace(-0.1, 1.1, 13),
        r"$v$": numpy.linspace(-0.5, 0.5, 21),
        r"$p$": numpy.linspace(-0.5, 0.5, 21),
        r"$\omega_z$": numpy.linspace(-3.0, 3.0, 13),
    }

    # contour line levels
    lvl2 = {
        r"$u$": numpy.linspace(0.0, 1.0, 6),
        r"$v$": numpy.linspace(-0.5, 0.5, 11),
        r"$p$": numpy.linspace(-0.5, 0.5, 11),
        r"$\omega_z$": numpy.linspace(-3.0, 3.0, 13),
    }

    # subplot locations
    locs = [(0, 0), (0, 1), (1, 0), (1, 1)]

    # data
    if casename == "PetIBM":
        coords, vals = read_petibm_snapshot(petibmdir, time)
    else:
        coords, vals = read_pinn_snapshot(workdir, casename, time)

    # figure
    fig, axs = pyplot.subplots(2, 2, sharex=True, sharey=True, figsize=(8, 6))

    if time == "steady":
        fig.suptitle(rf"Cylinder flow, $Re=40$, steady-state")
    else:
        fig.suptitle(rf"Cylinder flow, $Re=40$, $t={time}$")

    for i, (field, val) in enumerate(vals.items()):
        ax = axs[locs[i]]
        ct1 = ax.contourf(*coords[field], val, lvl1[field], cmap="cividis", extend="both")
        ct2 = ax.contour(*coords[field], val, lvl2[field], colors='black', linewidths=0.5)
        ax.clabel(ct2, lvl2[field], fmt="%1.1f", inline_spacing=0.25, fontsize="small")
        ax.add_artist(pyplot.Circle((0., 0.), 0.5, color="w", zorder=10))

        ax.set_title(field)
        ax.set_xlim(-3, 4)
        ax.set_ylim(-2.5, 2.5)
        ax.set_aspect("equal", "box")

        fmt1 = ticker.ScalarFormatter(useOffset=True, useMathText=True)
        fmt1.set_powerlimits((0, 0))
        cbar = fig.colorbar(ct1, ax=ax, ticks=lvl2[field], format=fmt1)
        cbar.ax.get_yaxis().set_offset_position("left")

    axs[1, 0].set_xlabel("x")
    axs[1, 1].set_xlabel("x")
    axs[0, 0].set_ylabel("y")
    axs[1, 0].set_ylabel("y")

    # save
    workdir.joinpath("figures", casename).mkdir(exist_ok=True)
    if time == "steady":
        fig.savefig(workdir.joinpath("figures", casename, f"contour.png"))
    else:
        fig.savefig(workdir.joinpath("figures", casename, f"contour-t{time}.png"))
    # pyplot.close(fig)

In [ ]:
# a widget to adjust which case at what time to show
slider = ipywidgets.FloatSlider(
    value=1.0, min=1.0, max=20.0, step=1.0,
    description="Time:", orientation="horizontal",
)

dropdown = ipywidgets.Dropdown(
    options=[
        "PetIBM",
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
    ],
    value="nl6-nn512-npts6400-steady-raw", description="Case:"
)

out = ipywidgets.interactive_output(
    plot_single_snapshot,
    {"workdir": ipywidgets.fixed(workdir), "casename": dropdown,
    "time": slider, "petibmdir": ipywidgets.fixed(petibmdir)}
)

ipywidgets.VBox([out, slider, dropdown])

In [ ]:
def three_cols_plot(cases):
    """Plot contours from three cases side by side.
    """

    assert len(cases) == 3

    translator = {
        "PetIBM": "PetIBM",
        "nl6-nn512-npts6400-steady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$" + "\n" + r"steady",
        "nl6-nn512-npts6400-steady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$" + "\n" + r"steady, SWA",
        "nl6-nn512-npts6400-unsteady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$" + "\n" + r"unsteady",
        "nl6-nn512-npts6400-unsteady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$" + "\n" + r"unsteady, SWA",
    }

    # colored contour levels
    lvl1 = {
        r"$u$": numpy.linspace(-0.1, 1.1, 13),
        r"$v$": numpy.linspace(-0.5, 0.5, 21),
        r"$p$": numpy.linspace(-0.5, 0.5, 21),
        r"$\omega_z$": numpy.linspace(-3.0, 3.0, 13),
    }

    # contour line levels
    lvl2 = {
        r"$u$": numpy.linspace(0.0, 1.0, 6),
        r"$v$": numpy.linspace(-0.5, 0.5, 11),
        r"$p$": numpy.linspace(-0.5, 0.5, 11),
        r"$\omega_z$": numpy.linspace(-3.0, 3.0, 13),
    }

    # data
    data = []
    for case in cases:
        if case == "PetIBM":
            data.append(read_petibm_snapshot(petibmdir, 20))
        else:
            data.append(read_pinn_snapshot(workdir, case, 20.0))

    # figure
    fig, axs = pyplot.subplots(4, 3, sharex=True, sharey=True, figsize=(8, 11.5))
    fig.suptitle(rf"Cylinder flow, $Re=40$, flow field comparison")

    for col, (case, (coords, vals)) in enumerate(zip(cases, data)):
        for row, (field, val) in enumerate(vals.items()):
            ax = axs[row, col]
            ct1 = ax.contourf(*coords[field], val, lvl1[field], cmap="cividis", extend="both")
            ct2 = ax.contour(*coords[field], val, lvl2[field], colors='black', linewidths=0.5)
            ax.clabel(ct2, lvl2[field], fmt="%1.1f", inline_spacing=0.25, fontsize="small")
            ax.add_artist(pyplot.Circle((0., 0.), 0.5, color="w", zorder=10))

            ax.set_title(translator[case])
            ax.set_xlim(-3, 4)
            ax.set_ylim(-2.5, 2.5)
            ax.set_aspect("equal", "box")

            if col == 2: # only the last column will have colorbars
                fmt1 = ticker.ScalarFormatter(useOffset=True, useMathText=True)
                fmt1.set_powerlimits((0, 0))
                cbar = fig.colorbar(
                    ct1, ax=axs[row, :], format=fmt1, orientation="horizontal",
                    fraction=0.05, aspect=60,
                )
                cbar.set_label(field)
                cbar.ax.get_yaxis().set_offset_position("left")
                cbar.set_ticks(lvl2[field])

    for row in range(4):
        axs[row, 0].set_ylabel("y")

    for col in range(3):
        axs[3, col].set_xlabel("x")

    # save
    workdir.joinpath("figures").mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", f"contour-comparison.png"))
    # pyplot.close(fig)

In [ ]:
three_cols_plot(["PetIBM", "nl6-nn512-npts6400-steady-swa", "nl6-nn512-npts6400-unsteady-swa"])

In [ ]:
def plot_force_coefficients(workdir, cases):
    """Plot a field of all cases at a single time.
    """
    copts = cycler("color", pyplot.cm.tab10.colors)()

    translater = {
        "nl6-nn512-npts6400-steady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, steady",
        "nl6-nn512-npts6400-steady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, steady, SWA",
        "nl6-nn512-npts6400-unsteady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, unsteady",
        "nl6-nn512-npts6400-unsteady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, unsteady, SWA",
    }

    fig, ax = pyplot.subplots(1, 1, figsize=(8, 6))
    fig.suptitle(r"Lift ($C_L$) and drag ($C_D$) coefficients, $Re=40$")

    # line and label holders
    lines = []
    labels = []

    # add PetIBM results
    petibm = numpy.loadtxt(workdir.joinpath("data", "petibm-forces.txt"), dtype=float)

    lines.append((
        ax.plot(petibm[:, 0], petibm[:, 1]*2, "k-", lw=1, alpha=0.8)[0],
        ax.plot(petibm[:, 0], petibm[:, 2]*2, "k--", lw=1, alpha=0.8)[0]
    ))
    labels.append("PetIBM")

    for case in cases:
        with h5open(workdir.joinpath("outputs", case).with_suffix(".h5"), "r") as h5file:
            if h5file.attrs["unsteady"]:
                cd = h5file["coeffs/cd"][...]
                cl = h5file["coeffs/cl"][...]
                times = h5file["coeffs/times"][...]
            else:
                cd = numpy.full_like(petibm[:, 0], float(h5file["coeffs/cd"][...]))
                cl = numpy.full_like(petibm[:, 0], float(h5file["coeffs/cl"][...]))
                times = petibm[:, 0]
        
        c = next(copts)
        lines.append((
            ax.plot(times, cd, lw=1, ls="-", alpha=0.8, **c)[0],
            ax.plot(times, cl, lw=1, ls="--", alpha=0.8, **c)[0]
        ))
        labels.append(translater[case])
    
    ax.set_xlim(0., 20.)
    ax.set_xlabel(r"$t$")

    ax.set_ylim(-0.5, 2.5)
    ax.set_ylabel("$C_D$ and $C_L$")

    ax.legend(
        lines, labels, handler_map={tuple: HandlerTuple(ndivide=None)},
        ncol=2, loc="upper center", bbox_to_anchor=(0.5, -0.15)
    )

    # save
    workdir.joinpath("figures").mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", "drag-lift-coeffs.png"))
    # pyplot.close(fig)

In [ ]:
choices = ipywidgets.SelectMultiple(
    options=[
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
    ],
    value=[
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-swa",
    ],
    rows=10,
    description="Cases",
)

out = ipywidgets.interactive_output(
    plot_force_coefficients,
    {"workdir": ipywidgets.fixed(workdir), "cases": choices}
)

display(ipywidgets.HBox([out, choices]))

In [ ]:
def plot_training_history(workdir, casename, ws):
    """Plot figures related to training loss.
    """
    # fixed cycling kwargs
    kwargs = \
        cycler("color", pyplot.cm.tab10.colors[:3]) + \
        cycler("label", ["Raw data", "Moving averaged", "Moving minimum"]) + \
        cycler("linewidth", [0.3, 1.5, 1.5])
    kwargs = kwargs()

    kwr = next(kwargs)
    kwa = next(kwargs)
    kwm = next(kwargs)

    excludes = None
    data = read_tensorboard_data(workdir.joinpath(casename), excludes)

    fig, axs = pyplot.subplots(2, 1, sharex=False, sharey=False, figsize=(8, 6))
    fig.suptitle(rf"2D Cylinder flow, $Re=40$")
    
    # against steps
    axs[0].set_title("Aggregated loss v.s. iterations")
    axs[0].semilogy(data.index, data.loss, alpha=0.3, **kwr)
    axs[0].semilogy(data.index, data.loss.rolling(window=ws).mean(), **kwa)
    axs[0].semilogy(data.index, data.loss.rolling(window=ws).min(), **kwm)
    axs[0].axvline(200000, lw=0.3, ls="--", c="k")
    axs[0].set_xlabel("Iteration")
    axs[0].set_ylabel("Aggregated loss")
    axs[0].legend(loc=0)
    
    # against runtime
    axs[1].set_title("Aggregated loss v.s. run time")
    if "run_time" in data.columns:
        run_time = data.run_time/3600
        axs[1].semilogy(run_time, data.loss, alpha=0.3, **kwr)
        axs[1].semilogy(run_time, data.loss.rolling(window=ws).mean(), **kwa)
        axs[1].semilogy(run_time, data.loss.rolling(window=ws).min(), **kwm)
        try:
            axs[1].axvline(run_time.loc[200000], lw=0.3, ls="--", c="k")
        except KeyError:
            pass
    axs[1].set_xlabel("Run time (hours)")
    axs[1].set_ylabel("Aggregated loss")
    axs[1].legend(loc=0)

    # save
    workdir.joinpath("figures", casename).mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", casename, f"loss-hist.png"))
    # pyplot.close(fig)

In [ ]:
# a widget to adjust which case at what time to show
slider = ipywidgets.IntSlider(
    value=10, min=1, max=200, step=2,
    description="Window size:", orientation="horizontal",
)

dropdown = ipywidgets.Dropdown(
    options=[
        "nl6-nn512-npts6400-steady",
        "nl6-nn512-npts6400-unsteady",
    ],
    value="nl6-nn512-npts6400-steady", description="Case:"
)

out = ipywidgets.interactive_output(
    plot_training_history,
    {"workdir": ipywidgets.fixed(workdir), "casename": dropdown, "ws": slider}
)

display(ipywidgets.VBox([out, slider, dropdown]))

In [ ]:
def read_validation_data(workdir):
    """Read validation data (surface pressure).
    """
    sen2009 = pandas.read_csv(
        workdir.joinpath("data", "sen_et_al_2009.csv"),
        header=0, names=["degrees", "values"]
    )
    sen2009["degrees"] = 180. - sen2009["degrees"]  # they defined 0 from left

    park1998 = pandas.read_csv(
        workdir.joinpath("data", "park_et_al_1998.csv"),
        header=None, names=["degrees", "values"]
    )
    park1998["degrees"] = 180. - park1998["degrees"]  # they defined 0 from left

    grove1964 = pandas.read_csv(
        workdir.joinpath("data", "grove_et_al_1964.csv"),
        header=0, names=["degrees", "values"]
    )
    grove1964["degrees"] = 180. - grove1964["degrees"]  # they defined 0 from left

    with h5open(workdir.joinpath("data", "petibm-probe-p.h5"), "r") as h5file:
        x = h5file["mesh/x"][...]
        y = h5file["mesh/y"][...]
        ids = numpy.argsort(h5file["mesh/IS"][...].flatten())
        p = h5file[f"p/{max(h5file['p'].keys(), key=float)}"][...].flatten()[ids].reshape(y.size, x.size)

        # initialize final dataframe
        petibm = pandas.DataFrame(data={
            "degrees": numpy.linspace(0., numpy.pi, 361),
            "values": numpy.zeros(361)
        })

        # create an interpolater for surface pressure from PetIBM
        probe_interp = scipy.interpolate.RectBivariateSpline(x, y, p.T)
        
        # note for PetIBM's diffused immersed boundary, we use r+3dx as the cylinder surface
        surfx = (0.5 + 3 * (x[1] - x[0])) * numpy.cos(petibm.degrees.array)
        surfy = (0.5 + 3 * (x[1] - x[0])) * numpy.sin(petibm.degrees.array)

        # interpolation
        petibm["values"] = probe_interp(surfx, surfy, grid=False) * 2  # Cp = (p-p_ref) * 2

        # convert from radius to degrees
        petibm["degrees"] = petibm["degrees"] * 180 / numpy.pi
    
    return {"sen2009": sen2009, "park1998": park1998, "grove1964": grove1964, "petibm": petibm}

In [ ]:
def plot_surface_pressure(workdir, cases, refs):
    """Plot surface pressure coefficients.
    """

    translater = {
        "nl6-nn512-npts6400-steady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, steady",
        "nl6-nn512-npts6400-steady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, steady, SWA",
        "nl6-nn512-npts6400-unsteady-raw": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, unsteady",
        "nl6-nn512-npts6400-unsteady-swa": r"$(N_l, N_n, N_{bs})=(6, 512, 6400)$, unsteady, SWA",
    }

    fig, ax = pyplot.subplots(1, 1, figsize=(8, 6))
    fig.suptitle(r"Pressure coefficients on cylinder surface, $Re=40$, $t=20$")

    # reference: petibm
    ax.plot(
        refs["petibm"]["degrees"], refs["petibm"]["values"],
        c="k", ls="-", lw=2, alpha=0.7,
        label="PetIBM"
    )

    kwargs = {"lw": 1.5, "alpha": 0.9, "ls": "--"}
    for case in cases:
        with h5open(workdir.joinpath("outputs", f"{case}.h5"), "r") as h5file:
            thetas = h5file["surfp/degrees"][...]
            values = h5file["surfp/cp"][...]

        ax.plot(thetas, values, label=translater[case], **kwargs)

    # other references
    ax.plot(
        refs["grove1964"]["degrees"], refs["grove1964"]["values"],
        ls="none", marker="s", ms=7, alpha=0.8,
        label="Grove et al., 1964",
    )

    ax.plot(
        refs["sen2009"]["degrees"], refs["sen2009"]["values"],
        ls="none", marker="o", mfc="none", ms=6, alpha=0.8,
        label="Sen et al., 2009"
    )

    ax.plot(
        refs["park1998"]["degrees"], refs["park1998"]["values"],
        ls="none", marker="^", mfc="none", ms=6, alpha=0.8,
        label="Park et al., 1998"
    )

    ax.set_xlabel(r"Degree from $+x$ axis")
    ax.set_xlim(0, 180.)

    ax.set_ylabel(r"Pressure coefficient, $C_p$")
    ax.set_ylim(-1.2, 1.5)

    ax.grid()
    ax.legend(loc=0)

    # save
    workdir.joinpath("figures").mkdir(exist_ok=True)
    fig.savefig(workdir.joinpath("figures", "surface-pressure.png"))
    # pyplot.close(fig)

In [ ]:
refs = read_validation_data(workdir)

choices = ipywidgets.SelectMultiple(
    options=[
        "nl6-nn512-npts6400-steady-raw",
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-raw",
        "nl6-nn512-npts6400-unsteady-swa",
    ],
    value=[
        "nl6-nn512-npts6400-steady-swa",
        "nl6-nn512-npts6400-unsteady-swa",
    ],
    rows=10,
    description="Cases",
)

out = ipywidgets.interactive_output(
    plot_surface_pressure,
    {"workdir": ipywidgets.fixed(workdir), "cases": choices, "refs": ipywidgets.fixed(refs)}
)

display(ipywidgets.HBox([out, choices]))